<a href="https://colab.research.google.com/github/JITHIN-ANTONY-JOSEPH/ERP_11358080/blob/main/3_Validation_Set_Recipe1mSubstitution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mounting to connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importing required libraries and bothe the datasets

In [ ]:
import pandas as pd
import json
import re

In [ ]:
flavorgraph_df = pd.read_csv('/content/drive/My Drive/ERP/Dataset/nodes_191120.csv')  # Adjust the path as needed , this is the path to my personal Google Drive


In [ ]:
# Load the Recipe1m dataset from the JSON file
with open('/content/drive/My Drive/ERP/modified_Processed_Layer1.json', 'r') as file:  # Adjust the path as needed , this is the path to my personal Google Drive
    recipe1m_data = [json.loads(line) for line in file]

### Core-logic to generate Recipe1MSubstitution Validation dataset

In [ ]:
import pandas as pd
import json
import re
from concurrent.futures import ProcessPoolExecutor

# Extract unique ingredients where node_type is "ingredient"
flavorgraph_ingredients = set(flavorgraph_df[flavorgraph_df['node_type'] == 'ingredient']['name'].dropna().unique())

# Substitution keywords
substitution_keywords = ["instead", "substitute", "in place of", "replace"]

# Convert to DataFrame for easier manipulation
recipe1m_df = pd.DataFrame(recipe1m_data)

# Function to extract substitution pairs from text
def extract_substitution_pairs(text, ingredients_set, keywords):
    pairs = []
    sentences = re.split(r'[.!?]', text)
    for sentence in sentences:
        # Check if any multi-word keyword exists in the sentence
        if any(keyword in sentence for keyword in keywords):
            words = sentence.split()
            for i in range(len(words)):
                for keyword in keywords:
                    keyword_len = len(keyword.split())
                    if ' '.join(words[i:i+keyword_len]) == keyword:
                        # Look 7 words to the left and right for ingredients
                        left_bound = max(i - 7, 0)
                        right_bound = min(i + keyword_len + 7, len(words))
                        surrounding_words = words[left_bound:i] + words[i+keyword_len:right_bound]
                        ingredient_positions = [(word, j) for j, word in enumerate(surrounding_words) if word in ingredients_set]
                        if len(ingredient_positions) >= 2:
                            for k in range(len(ingredient_positions) - 1):
                                ing1 = ingredient_positions[k][0]
                                ing2 = ingredient_positions[k + 1][0]
                                if keyword != "replace":
                                    pairs.append((ing2, ing1))
                                else:
                                    pairs.append((ing1, ing2))
                        break
    return pairs

# Function to process a specific recipe row
def process_recipe_row(row, ingredients_set, keywords):
    substitution_pairs = []
    if isinstance(row['processed_instructions'], list):
        instructions = ' '.join(row['processed_instructions'])
        pairs = extract_substitution_pairs(instructions, ingredients_set, keywords)
        substitution_pairs.extend(pairs)
    return substitution_pairs

# Function to process a batch of recipes
def process_batch(batch, ingredients_set, keywords):
    batch_substitution_pairs = []
    for index, row in batch.iterrows():
        pairs = process_recipe_row(row, ingredients_set, keywords)
        for pair in pairs:
            batch_substitution_pairs.append((pair[0], pair[1], row['id'], row['partition']))
    return batch_substitution_pairs

# Split the DataFrame into batches for parallel processing
num_batches = 10  # Adjust this number based on your system's capabilities
batches = np.array_split(recipe1m_df, num_batches)

# Process the batches in parallel
substitution_pairs = []
with ProcessPoolExecutor() as executor:
    results = list(executor.map(process_batch, batches, [flavorgraph_ingredients]*num_batches, [substitution_keywords]*num_batches))
    for result in results:
        substitution_pairs.extend(result)

# Convert the substitution pairs to a DataFrame
substitution_pairs_df = pd.DataFrame(substitution_pairs, columns=['ingredient1', 'ingredient2', 'recipe_id', 'partition'])

# Drop duplicate rows
substitution_pairs_df = substitution_pairs_df.drop_duplicates()

# Remove cases where ingredient1 equals ingredient2
substitution_pairs_df = substitution_pairs_df[substitution_pairs_df['ingredient1'] != substitution_pairs_df['ingredient2']]

# Save the substitution pairs to a CSV file
substitution_pairs_df.to_csv('/content/drive/My Drive/ERP/Recipe1MSubs_full.csv', index=False) # Adjust the path as needed , this is the path to my personal Google Drive

# Display the first few rows of the resulting DataFrame
print(substitution_pairs_df.head())

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


  ingredient1 ingredient2   recipe_id partition
0        beef       liver  000b0b5d50     train
1  vermicelli    couscous  000b6f9852     train
2      orange       peach  0013894a19       val
3  strawberry      orange  0013894a19       val
4      orange  strawberry  0013894a19       val


### Generating counts by partition

In [ ]:
# Get the count of entries based on partition
partition_counts = substitution_pairs_df['partition'].value_counts()

# Display the counts
print(partition_counts)

partition
train    13636
val       3008
test      2899
Name: count, dtype: int64
